In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from craftax.craftax_env import make_craftax_env_from_name
import jax
import numpy as np
import jax.numpy as jnp
from functools import partial
from craftax.craftax.renderer import render_craftax_pixels
from craftax.craftax import constants
import matplotlib.pyplot as plt
import craftax_env
from craftax.craftax.constants import Action

Loading textures from cache.
Textures successfully loaded from cache.


In [ ]:
from craftax_env import CraftaxSymbolicEnvNoAutoReset
from jaxneurorl.wrappers import TimestepWrapper

render = jax.jit(partial(render_craftax_pixels, block_pixel_size=constants.BLOCK_PIXEL_SIZE_IMG))
def render_craftax(state):
    image = render_craftax_pixels(state, block_pixel_size=constants.BLOCK_PIXEL_SIZE_IMG)
    return image/255.0
    
render_craftax = jax.jit(render_craftax)


env = TimestepWrapper(CraftaxSymbolicEnvNoAutoReset(), False)
env_params = env.default_params.replace(
    mob_despawn_distance=3,
    day_length=3,
    reset_seeds=jnp.arange(2, 4),
    max_timesteps=3,
)

seed = 16
rng = jax.random.PRNGKey(seed)

timestep = env.reset(rng, env_params)
image = render_craftax(timestep.state)
plt.imshow(image)
plt.show()
possible = craftax_env.get_possible_achievements(timestep.state)
craftax_env.print_possible_achievements(possible)


/Users/wilka/miniforge3/envs/preplay-ai/lib/python3.10/site-packages/jax/_src/core.py:700: FutureWarning: unhashable type: <class 'jax._src.interpreters.partial_eval.DynamicJaxprTracer'>. Attempting to hash a tracer will lead to an error in a future JAX release.
  warnings.warn(


In [ ]:
for _ in range(10):
    print('-'*10)
    rng, rng_ = jax.random.split(rng)
    action = env.action_space().sample(rng_)
    timestep = env.step(rng, timestep, action, env_params)
    image = render_craftax(timestep.state)
    plt.imshow(image)
    plt.show()
    print(timestep.last())
    possible = craftax_env.get_possible_achievements(timestep.state)
    craftax_env.print_possible_achievements(possible)
